In [ ]:
from Q6_custom_environment_99210283 import custom
from torch.distributions import Normal
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super (Policy, self).__init__()
        self.FC = nn.Sequential (nn.Linear (2, 32), nn.Dropout(0.5), nn.ReLU(),
                                  nn.Linear (32, 32), nn.Dropout(0.5), nn.ReLU(),
                                  nn.Linear (32, 2))
        
    def forward(self, state):
        return self.FC (torch.tensor([state[0], state[1]]))

In [ ]:
optimizer = torch.optim.Adam(policy.parameters(), lr = learning_rate)
learning_rate = 1e-3
gamma = 0.9
num_episodes= 10000
num_max_step= 200
policy = Policy()
reward_history = []
loss_history = []
policy_history = []
reward_episode = []
env = custom()

for episode in range(num_episodes):
    state = env.Res()
    for step in range(num_max_step):
        mu, var = policy (state)
        action_dist = Normal(mu, var)
        action = action_dist.sample()
        new_state, reward, done = env.Step(action)
        state = torch.tensor (new_state)
        reward_episode.append(reward)
        policy_history.append(action_dist.log_prob(action)[None])

    print(f'final state is {state}')
    return_rewards=[]
    
    disc_reward = 0
    for i,r in enumerate (reward_episode[::-1]):
        
        disc_reward = r + gamma * disc_reward
        return_rewards.append (disc_reward)
    return_rewards = torch.tensor (return_rewards).flip(dims = [-1])
    

    return_rewards = (return_rewards - return_rewards.mean()) / (return_rewards.std() + 1e-8)
    
    v = torch.cat (policy_history)
    
    loss = -(return_rewards * v).sum()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss = loss.item()
    
    loss_history.append(loss)
    reward_history.append(np.sum(reward_episode)/step)

    print('[Episode %d / %d] reward: %.3f' %
                  (episode, num_episodes, np.sum(reward_episode)/step))

    policy_history = []
    reward_episode= []

 
